<a href="https://colab.research.google.com/github/achrip/fictional-octo-broccoli/blob/main/embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq \
langchain langchain-community langchain-huggingface langchain-chroma \
pypdf transformers accelerate Xformers InstructorEmbedding \
sentencepiece bitsandbytes tiktoken chromadb typer semantic_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Document Pre-Processing

- Importing PDFs
- Parsing PDFs
- Splitting into chunks

In [ ]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from semantic_split import SimilarSentenceSplitter, SentenceTransformersSimilarity, SpacySentenceSplitter

In [ ]:
path = "/content/drive/MyDrive/ncvs_documents/"
loader = DirectoryLoader(path=path,
                         glob="*.pdf",
                         loader_cls=PyPDFLoader)
documents = loader.load()

The `documents` is specifically an instance of `List[Document]` class. It consists of the tuple: `page_content` and `metadata` as shown below.

In [ ]:
documents[36]

Document(page_content='Chapter VIII Manning Bab VIII  Pengawakan NCVS Indonesia\nVIII - 379.3.10. 2nd Class Engineer \n   c. tugas yang berkaitan dengan penyerahan  \n\t \t \t \t dinas\tjaga\tprosedur\tkeselamatan\tdan\t\t \t\n    darurat; mengubah semua sistem remote  \n    control;\n   kewaspadaan keselamatan yang harus dilaku\n\t \t \t kan\tselama\tdinas\tjaga\tdan\ttindakan\tsegera\t\t\n\t \t \t yang\tharus\tdilaksanakan\tjika\tterjadi\tkeba-\n   karan atau kecelakaan, khususnya yang   \n   berkaitan dengan sistem bahan bakar.\n4) Mengoperasikan Sistem pompa: \n sistem pompa dan a. operasi pemompaan rutin \n sistem pengendali b. operasi sistem pompa bilga, ballast dan \n terkait.  muatan\nPengendalian operasional kapal dan kepedulian terhadap kesela-\nmatan orang di atas kapal pada tingkat operasional\n5) memastikan peme- Pencegahan pencemaran lingkungan laut\n nuhan persyaratan dan prosedur anti pencemaran \n pencegahan pence-\n maran  Pengetahuan mengenai kewaspadaan yang  \n   h

## Splitting into Chunks

There are two such splitters that are recommended here, one is the `RecursiveCharacterTextSplitter`from Langchain and the other is the [`semantic-split`](https://github.com/agamm/semantic-split) by Agamm.

> Known issues:  
> `semantic-split` uses string as arguments, meaning we _could_ very well lose page metadata of the content itself. Workaround is in progress.

In [ ]:
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=512,
                                                   chunk_overlap=64)
semantic_splitter = SimilarSentenceSplitter(similarity_model=SentenceTransformersSimilarity(),
                                           sentence_splitter=SpacySentenceSplitter())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
recursive_text = recursive_splitter.split_documents(documents)

semantic_text= list()
for i in range(len(documents)):
  semantic_text.append(semantic_splitter.split(documents[i].page_content))

# Export to ChromaDB

Embeddings created will be stored in ChromaDB, and Chroma can also act like a retriever by utilizing an embedding model.

In [ ]:
from langchain_chroma import Chroma

